# **orders data stage load**

This notebook orders data from source files and loads it into the staging table 

In [0]:
# Configuration
source_dir = "/Volumes/event-driven-catalog/default/incremental_load/orders_data/source/"
archive_dir = "/Volumes/event-driven-catalog/default/incremental_load/orders_data/archive/"
stage_table = "`event-driven-catalog`.default.order_stage"

print(f"Processing orders data from: {source_dir}")
print(f"Staging table: {stage_table}")

Processing orders data from: /Volumes/event-driven-catalog/default/incremental_load/orders_data/source/
Staging table: `event-driven-catalog`.default.order_stage


In [0]:
# Import required libraries
from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import datetime
import json

# Define schema for orders data
orders_schema = StructType([
    StructField("order_id", StringType(), False),
    StructField("customer_id", StringType(), False),
    StructField("product_id", StringType(), False),
    StructField("order_date", DateType(), False),
    StructField("order_amount", DecimalType(10,2), False),
    StructField("currency", StringType(), False),
    StructField("payment_method", StringType(), False),
    StructField("shipping_address", StringType(), False),
    StructField("order_status", StringType(), False),
    StructField("created_timestamp", TimestampType(), False)
])

print("Schema defined for orders data")


Schema defined for orders data


In [0]:
# Read and validate orders data
try:
    # Read CSV files with schema validation
    df_orders = spark.read.schema(orders_schema).csv(source_dir, header=True, dateFormat="yyyy-MM-dd", timestampFormat="yyyy-MM-dd HH:mm:ss")
    
    # Add processing metadata
    df_orders = df_orders.withColumn("processed_timestamp", F.current_timestamp()) \
                        .withColumn("batch_id", F.lit(datetime.now().strftime("%Y%m%d_%H%M%S"))) \
                        .withColumn("source_system", F.lit("ecommerce_orders"))
    
    # Data quality checks

    total_records = df_orders.count()
    null_order_ids = df_orders.filter(F.col("order_id").isNull()).count()
    null_customer_ids = df_orders.filter(F.col("customer_id").isNull()).count()
    invalid_amounts = df_orders.filter(F.col("order_amount") <= 0).count()
    
    print(f"Total records processed: {total_records}")
    print(f"Records with null order_id: {null_order_ids}")
    print(f"Records with null customer_id: {null_customer_ids}")
    print(f"Records with invalid amounts: {invalid_amounts}")

    # Filter out valid records - Fixed boolean logic
    df_valid_orders = df_orders.filter(
        (F.col("order_id").isNotNull()) & 
        (F.col("customer_id").isNotNull()) & 
        (F.col("order_amount") > 0)
    )

    # Capture invalid records for error handling - Fixed boolean logic
    df_invalid_orders = df_orders.filter(
        (F.col("order_id").isNull()) | 
        (F.col("customer_id").isNull()) | 
        (F.col("order_amount") <= 0)
    )
    
    valid_records = df_valid_orders.count()
    invalid_records = df_invalid_orders.count()
    
    print(f"Valid records: {valid_records}")
    print(f"Invalid records: {invalid_records}")

except Exception as e:
    print(f"Error reading orders data: {str(e)}")
    raise



Total records processed: 20
Records with null order_id: 0
Records with null customer_id: 0
Records with invalid amounts: 0
Valid records: 20
Invalid records: 0


In [0]:
# Write valid data to staging table
try:
    # Create or overwrite staging table
    df_valid_orders.write.format("delta").mode("overwrite").saveAsTable(stage_table)
    print(f"Successfully loaded {valid_records} valid orders to staging table")

except Exception as e:
    print(f"Error writing to staging table: {str(e)}")
    raise


Successfully loaded 20 valid orders to staging table


In [0]:
# Archive processed files
try:
    # List all files in the source directory
    files = dbutils.fs.ls(source_dir)
    
    archived_count = 0
    for file in files:
        if file.name.endswith('.csv'):
            src_path = file.path
            archive_path = archive_dir + file.name
            
            # Move the file to archive
            dbutils.fs.mv(src_path, archive_path)
            archived_count += 1
            print(f"Archived: {file.name}")
    
    print(f"Successfully archived {archived_count} files")
    
except Exception as e:
    print(f"Error archiving files: {str(e)}")
    raise

Archived: orders_2025_01_15.csv
Successfully archived 1 files
